# Experiment 0: Rerun from paper source code

The source notebooks were run on [Kaggle][source] by basically running some of the [codes][paper-repo] on the original [paper] on Synaptic metaplasticity in BNN (with some modifications).

Then I downloaded data to `data/output/exp0-pmnist-paper-rerun`


- `data/output/exp0-pmnist-paper-rerun/vary-meta`: [Version 1][V1] of [source]
  - variation of `meta` parameter: `[0.0, 0.5, 1.0, 1.35, 2]`
  - these following parameters were fixed:

    ``` bash
    --hidden-layers 1024 1024 --lr 0.005 --decay 1e-7 --epochs-per-task 25
    --task-sequence 'pMNIST' 'pMNIST' 'pMNIST' 'pMNIST' 'pMNIST'
    ```

  - data downloaded from `SynapticMetaplasticityBNN/Continual_Learning_Fig-2abcdefgh-3abcd-5cde/results` of [V1] (except for `16-01-01_gpu0`)
- `data/output/exp0-pmnist-paper-rerun/vary-hidden`: [Version 2][V2] of [source]
  - variation of size of hidden layers: `[512, 1024, 2048, 4096]`
  - these following parameters were fixed:

    ``` bash
    --meta 1.35 --lr 0.005 --decay 1e-7 --epochs-per-task 20
    --task-sequence 'pMNIST' 'pMNIST' 'pMNIST' 'pMNIST' 'pMNIST'
    ```

  - data downloaded from `SynapticMetaplasticityBNN/Continual_Learning_Fig-2abcdefgh-3abcd-5cde/results/2021-11-04` of [V2]

[paper-repo]: https://github.com/Laborieux-Axel/SynapticMetaplasticityBNN
[paper]: https://www.nature.com/articles/s41467-021-22768-y
[source]: https://www.kaggle.com/penguinsfly/bnn-meta-rerun-from-paper
[V1]: https://www.kaggle.com/penguinsfly/bnn-meta-rerun-from-paper/data?scriptVersionId=78778974
[V2]: https://www.kaggle.com/penguinsfly/bnn-meta-rerun-from-paper/data?scriptVersionId=78817005



In [1]:
%%capture
!pip install --upgrade plotly
!pip install -U kaleido

In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust


In [3]:
import os, glob 
import pandas as pd 
import numpy as np 
import yaml 
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go

In [4]:
data_root = Path('data/output/exp0-pmnist-paper-rerun')
fig_root = Path('figures/exp0-pmnist-paper-rerun')
fig_root.mkdir(exist_ok=True)

In [5]:
def load_df(data_root, data_dir, prm_fns = None):
    data_path = data_root / data_dir
    csv_file = glob.glob(str(data_path / '*.csv'))[0]
    prm_file = data_path / 'hyperparameters.txt'

    hyp_params = yaml.safe_load(open(prm_file))
    hyp_params = {k:v for list_item in hyp_params for (k,v) in list_item.items()}

    prm2save = {k: fn(hyp_params) for k, fn in prm_fns.items()} if prm_fns else dict()

    df = pd.read_csv(csv_file)\
            .filter(regex='(acc_test_.*|task_order|epoch)', axis=1)\
            .assign(**prm2save)\
            .melt(id_vars=['task_order','epoch'] + list(prm2save.keys()), 
                var_name='test_set', value_name='test_acc')
            
    df['test_set'] = df['test_set'].apply(lambda x: 'task-' + x[-1])

    max_epoch = max(df.epoch)
    df.epoch = (df.task_order - 1) * (max_epoch) + df.epoch 
    return df

def load_dfs(data_root, prm_fns=None, concat=True):
    df = [load_df(data_root, d, prm_fns) 
          for d in os.listdir(data_root)]
    if concat:
        df = pd.concat(df, ignore_index=True)
    return df


In [6]:
axis_config = dict(
    showline=True,
    showgrid=False,
    showticklabels=True,
    linecolor='rgb(0, 0, 0)',
    linewidth=2,    
    ticks='outside',
    tickwidth=2
    )

font_config = dict(
    family="Fira Sans",
    size=15,
    color='black'
    )

title_config = dict(
    title_x = 0.5,
    title_y = 0.95,
    title_xanchor = 'center',
    title_yanchor = 'top',
    title_font_size=20
)

general_layout = go.Layout(
    xaxis=axis_config,
    yaxis=axis_config,
    font=font_config,
    margin=dict(autoexpand=True,l=100,r=50,t=100,b=120),
    showlegend=True,
    plot_bgcolor='white',
    autosize=True,
    **title_config
)

for i in range(1,10): # for subplots
    general_layout['xaxis' + str(i)] = axis_config
    general_layout['yaxis' + str(i)] = axis_config

# Vary meta

In [7]:
prm_fns = {
    'meta': lambda hyp_prm: hyp_prm['meta'][0],
    'num_hidden': lambda hyp_prm: hyp_prm['hidden layers'][0]
}

df = load_dfs(data_root / 'vary-meta', prm_fns)


## Plot progress

In [ ]:
fig = px.line(
    df, x="epoch", y="test_acc", 
    color="test_set",
    facet_col="meta",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    facet_col_wrap=2, 
    facet_col_spacing=0.05,
    facet_row_spacing=0.1
    )

fig.update_layout(
    general_layout,
    width=1800,
    height=800,
    title_text='Variation of meta (1024 x 1024)'
)

fig.write_image(fig_root / 'vary-meta-progress.svg')

fig.show()

## Plot final acc

In [9]:
max_globepoch = max(df.epoch)
final_df = df.query('epoch == @max_globepoch')\
    .filter(['meta','test_set', 'test_acc'])\
    .reset_index(drop=True)\
    .rename(columns={'test_acc': 'final test acc'})
final_df['test_set'] = final_df['test_set'].apply(lambda x: int(x[-1]))


In [10]:
fig = px.line(
    final_df, x="test_set", y="final test acc", color="meta",
    color_discrete_sequence=px.colors.sequential.Burg,
    markers=True
)

fig.update_traces(line=dict(width=4), marker_size=20)

fig.update_layout(
    general_layout,
    height=500, width=700,
    title_y = 0.9,
    font_size = 20, title_font_size = 25,
    title_text = 'Variations of meta (1024 x 1024)'
)

fig.write_image(fig_root / 'vary-meta-final.svg')

fig.show()

# Vary number of hidden units per layer

In [11]:
df = load_dfs(data_root / 'vary-hidden', prm_fns)

## Plot progess

In [ ]:
fig = px.line(
    df, x="epoch", y="test_acc", 
    color="test_set",
    facet_col="num_hidden",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    facet_col_wrap=2, 
    facet_col_spacing=0.05,
    facet_row_spacing=0.1
    )

fig.update_layout(
    general_layout,
    width=1800,
    height=800,
    title_text='Variation of number of hidden layer units (meta = 1.35)'
)

fig.write_image(fig_root / 'vary-hidden-progress.svg')

fig.show()

## Plot final acc

In [13]:
max_globepoch = max(df.epoch)
final_df = df.query('epoch == @max_globepoch')\
    .filter(['num_hidden','test_set', 'test_acc'])\
    .reset_index(drop=True)\
    .rename(columns={'test_acc': 'final test acc'})
final_df['test_set'] = final_df['test_set'].apply(lambda x: int(x[-1]))


In [14]:
fig = px.line(
    final_df, x="test_set", y="final test acc", color="num_hidden",
    color_discrete_sequence=px.colors.sequential.Burg,
    markers=True, labels = dict(num_hidden = '# units')
)

fig.update_traces(line=dict(width=4), marker_size=20)

fig.update_layout(
    general_layout,
    height=500, width=700,
    title_y = 0.9,
    font_size = 20, title_font_size = 25,
    title_text = 'Variations of # hidden units per layer (meta = 1.35)'
)

fig.write_image(fig_root / 'vary-hidden-final.svg')

fig.show()